In [1]:
import dask.dataframe as dd
import pandas as pd
from dask.multiprocessing import get
import random

In [2]:
df = pd.DataFrame({'col_1':random.sample(range(100), 100), 'col_2': random.sample(range(100), 100) })

In [3]:
df.head()

,col_1,col_2
0,29,70
1,4,32
2,24,41
3,13,87
4,8,0


In [11]:
def test_f(df, col_1, col_2):
    return df.assign(result=df[col_1] * df[col_2])

In [4]:
# def test_f(col_1, col_2):
#     return col_1*col_2

In [16]:
ddf = dd.from_pandas(df, npartitions=2)

In [17]:
ddf_out = ddf.map_partitions(test_f, 'col_1', 'col_2')

In [18]:
ddf_out.compute() 

,col_1,col_2,result
0,29,70,2030
1,4,32,128
2,24,41,984
3,13,87,1131
4,8,0,0
5,93,12,1116
6,18,9,162
7,75,95,7125
8,11,81,891
9,45,99,4455


In [6]:
#res = ddf.map_partitions(test_f)

In [7]:
#tt = ddf.map_partitions(test_f, columns=['col_1', 'col_2']).compute()

In [19]:
dds_out = ddf.apply(
    test_f, 
    args=('col_1', 'col_2'), 
    axis=1, 
    meta=('result', int)
).compute()  

AttributeError: ("'Series' object has no attribute 'assign'", 'occurred at index 50')

In [8]:
ddf['result'] = ddf.map_partitions(test_f, columns=['col_1', 'col_2']).compute(get=get)

ValueError: Metadata inference failed in `test_f`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
TypeError("test_f() got an unexpected keyword argument 'columns'",)

Traceback:
---------
  File "c:\programdata\psc_env\lib\site-packages\dask\dataframe\utils.py", line 160, in raise_on_meta_error
    yield
  File "c:\programdata\psc_env\lib\site-packages\dask\dataframe\core.py", line 3683, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))


In [20]:
import dask.dataframe as dd
import pandas as pd
from dask.multiprocessing import get
import random

df = pd.DataFrame({'col_1':random.sample(range(10000), 10000), 'col_2': random.sample(range(10000), 10000) })

def test_f(col_1, col_2):
    return col_1*col_2

ddf = dd.from_pandas(df, npartitions=8)

In [22]:
ddf.map_partitions(lambda df: df.assign(z=df.col_1 * df.col_2))

,col_1,col_2,z
npartitions=8,,,
0,int64,int64,int64
1250,...,...,...
...,...,...,...
8750,...,...,...
9999,...,...,...


In [23]:
def test_f(df, col_1, col_2):
    return df.assign(result=df[col_1] * df[col_2])

In [24]:
ddf_out = ddf.map_partitions(test_f, 'col_1', 'col_2')

In [26]:
# Here is good place to do something with BIG ddf_out dataframe before calling .compute()
result = ddf_out.compute(cheduler='processes')  # Will load the whole dataframe into memory

In [27]:
result

,col_1,col_2,result
0,7756,5446,42239176
1,9144,2239,20473416
2,3844,1322,5081768
3,9205,1443,13282815
4,1547,4056,6274632
5,7815,7362,57534030
6,6533,9121,59587493
7,3386,9253,31330658
8,5685,5185,29476725
9,1104,5187,5726448
